Kalman Filter Introduction
~~~~~~~~~~~~~~~~~~~~~~~~~~

In [23]:
# Library imports
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

The Kalman filter is a predictive approach to state estimation. In short, the Kalman filter predicts what a measurement should be based on previous measurements and the model of the system. Then, the filter will use new measurements to update the prediction. Finally, the filter will assess the accuracy of the model and the sensor. These steps are repeated to converge to the true state.

Hear are some resources about Kalman filters. Skim over these for a brief introduction to the theory:

https://en.wikipedia.org/wiki/Kalman_filter

https://web.mit.edu/kirtley/kirtley/binlustuff/literature/control/Kalman%20filter.pdf

We will consider a simplified version of our air brake control problem. Consider a point mass moving upwards with zero acceleration. Then, we get the following equations of motion:

    v(t) = v_0
    y(t) = y_0 + v_0*t

We will assume that we have sensors to measure y--our barometers; however, we cannot measure v directly. Additonally, our barometer is not 100% accurate nor precise. It exhibits some random noise in it's measurements and some random calibration error.

In [24]:
# ~~~~~~ Problem Setup ~~~~~~ 

# We include some randomness in the initialization to model uncertainty.
# You can set a seed to get the same randomness everytime. To do so, uncomment the next line:
# rd.seed(2024)

# Class to model motion
class MotionState:
    # State Initialization
    def __init__(self, y_0, v_0, timestep, sensor_var):
        # Initial Conditions
        self.y_0 = y_0
        self.v_0 = v_0
        self.y = y_0
        self.v = v_0

        # Sensor Error
        self.sensor_var = sensor_var
        self.calibration_error = rd.gauss(0.0, sensor_var)

        # System Computation Timestep
        self.timestep = timestep

    # State advance
    def advance_time(self):
        self.y = self.y + self.timestep*self.v
        self.v = self.v

    # Read the barometer
    def measure_y(self):
        return rd.gauss(self.y, self.sensor_var) + self.calibration_error

Next, we will use a Kalman Filter to estimate the position and velocity of an object as time advances. We will use the FilterPy library for this. See https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html for more information.

In [25]:
# TODO: Follow the example in the link to set up the Kalman Filter object.


Now, we will create an object in motion, and see how the Kalman Filter does with measuring the state.

In [26]:
# Create our rocket in motion
rocket = MotionState(y_0 = rd.randrange(5000, 10000), v_0 = rd.randrange(100, 250), timestep = 0.1, sensor_var = 0.1)

In [ ]:
# Read the barometer sensor
z = rocket.measure_y()
# Estimate the state using the Kalman Filter loop
f.predict()
f.update(z)

print("Height estimate is ", f.x[0])
print("True height is ", rocket.y)
print("Velocity estimate is ", f.x[1])
print("True velocity is ", rocket.v)

We will see how the estimates fair over time.

In [ ]:
# Initialize arrays to hold states
y_estimates = [f.x[0]]
v_estimates = [f.x[1]]
y_trues = [rocket.y]
v_trues = [rocket.v]
array_size = 15

for i in range(array_size - 1):
    rocket.advance_time()

    # Read the barometer sensor
    z = rocket.measure_y()
    # Estimate the state using the Kalman Filter loop
    f.predict()
    f.update(z)

    # Fill arrays
    y_estimates.append(f.x[0])
    v_estimates.append(f.x[1])
    y_trues.append(rocket.y)
    v_trues.append(rocket.v)

In [ ]:
# Graph the estimates
times = np.linspace(start = 0.0, stop = array_size * rocket.timestep, num = array_size)
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Height', color=color)
ax1.plot(times, y_estimates, color=color, label="Y Estimate", linestyle = "dashed")
ax1.plot(times, y_trues, color=color, label="Y True")
ax1.tick_params(axis='y', labelcolor=color)
plt.legend(loc='upper center')

ax2 = ax1.twinx()  # instantiate a second Axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Velocity', color=color)  # we already handled the x-label with ax1
ax2.plot(times, v_estimates, color=color, label="V Estimate", linestyle = "dashed")
ax2.plot(times, v_trues, color=color, label="V True")
ax2.tick_params(axis='y', labelcolor=color)
plt.legend(loc='center right')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

Experiment with changing the sensor variatoin, timestep, and covariance matrix, process uncertainty, and measurement uncertainty. How do these values change the settling time of the Kalman Filter predicitons? (Be sure to rerun the kalman filter creation cell so the matrices reset). 